### Write modified functions here and later add to main file

## DEREPLICATION Modification

In [1]:
input_dir <- "/Users/mahnoorzulfiqar/Downloads/MAW-main"

In [2]:
# ---------- Preparations ----------
# Load Libraries
library(Spectra)
library(MsBackendMgf)
library(MsBackendHmdb)
library(MsCoreUtils)
library(MsBackendMsp)
library(readr)
library(dplyr)
# 3 dependencies for latest MassBank version
library(rvest)
library(stringr)
library(xml2)
options(warn=-1)
#library("CAMERA")

Lade nötiges Paket: S4Vectors

Lade nötiges Paket: stats4

Lade nötiges Paket: BiocGenerics


Attache Paket: ‘BiocGenerics’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    IQR, mad, sd, var, xtabs


Die folgenden Objekte sind maskiert von ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min



Attache Paket: ‘S4Vectors’


Die folgenden Objekte sind maskiert von ‘package:base’:

    I, expand.grid, unname


Lade nötiges Paket: BiocParallel

Lade nötiges Paket: ProtGenerics


Attache Paket: ‘ProtGenerics’


Das folgende Objekt ist maskiert ‘package:stats’:

    smooth



Attache Paket: ‘Spectra’


Das folgende Objekt ist m

In [3]:
# load the functions file
source(file = paste(input_dir, "/Workflow_R_Functions.r", sep = ''))

In [4]:
input_table <- read.csv(file = paste(input_dir, "/input_table.csv", sep = ''))
input_table

X,mzml_files,ResultFileNames,File_id
<int>,<chr>,<chr>,<chr>
1,./VN_211016_Sc_QC_PRM_neg.mzML,./VN_211016_Sc_QC_PRM_neg,file_1
2,./VN_211016_Sc_QC_PRM_pos.mzML,./VN_211016_Sc_QC_PRM_pos,file_2
3,./VN_211016_Sc_st_PRM_neg.mzML,./VN_211016_Sc_st_PRM_neg,file_3
4,./VN_211016_Sc_st_PRM_pos.mzML,./VN_211016_Sc_st_PRM_pos,file_4
5,./VN_211016_acetyl_carnitine.mzML,./VN_211016_acetyl_carnitine,file_5
6,./VN_211016_betaine.mzML,./VN_211016_betaine,file_6
7,./VN_211016_butanoyl_carnitine.mzML,./VN_211016_butanoyl_carnitine,file_7
8,./VN_211016_cyst_acid_PRM_neg.mzML,./VN_211016_cyst_acid_PRM_neg,file_8
9,./VN_211016_cyst_acid_PRM_pos.mzML,./VN_211016_cyst_acid_PRM_pos,file_9


In [5]:
# parameters
mzml_file <- input_table[5, "mzml_files"]
pre_tbl <- paste(input_dir, str_remove(paste(input_table[5, "ResultFileNames"], "/premz_list.txt", sep = ""), "."), sep ="")
proc_mzml <- paste(input_dir, str_remove(paste(input_table[5, "ResultFileNames"], "/processedSpectra.mzML", sep = ""), "."), sep ="")
ppmx <- 15
result_dir <- input_table[5, "ResultFileNames"]
tbl <- read.table(pre_tbl)
pre_mz <- tbl[[1]]
x <- pre_mz

In [6]:
db <- "gnps"
file_id <- input_table[5, "File_id"]

In [7]:
# if the database selected is GNPS or all
if (db == "all" || db =="gnps"){

    # load the gnps spectral database
    load(file = paste(input_dir,"/gnps.rda", sep = ""))
}

In [20]:
spec_dereplication <- function(mzml_file, pre_tbl, proc_mzml, db, result_dir, file_id, input_dir, ppmx, error = TRUE){

    # read spectra object
    sps_all <- Spectra(proc_mzml, backend = MsBackendMzR())

    # extract precursor m/z
    tbl <- read.table(pre_tbl)
    pre_mz <- tbl[[1]]

    # common feature information

    id_X <- c() # id
    premz <- c() # precursor mz
    rtmin <- c() # stores rtmin
    rtmax <- c() # stores rtmax
    rtmed <- c() # stores calculated median of rtmin and rtmax
    rtmean <- c() # stores calculated mean of rtmin and rtmax
    col_eng <- c() # stores collision energy
    pol <- c() # stores polarity
    int <- c()
    source_file <- c() # source file
    nx <- c() # numbering the ids


    # for each pre mass
    for (x in pre_mz){

        # to name the file
        nx <- nx+1

        # filter spectra based on precusror m/z
        # this is done to extract all common information for id_X
        spsrt <- filterPrecursorMzRange(sps_all, x)

        # id based on file id, 
        id_Xx <- paste(file_id,  "M",  as.character(round(x, digits = 0)), 
                        "R", as.character(round(median(spsrt$rtime, na.rm = TRUE), digits = 0)), 
                        "ID", as.character(nx), sep = '')
        id_X <- c(id_X, id_Xx)

        pre <- x
        premz <- c(premz, pre)

        rti <- min(spsrt$rtime)
        rtmin <- c(rtmin, rti)

        rtx <- max(spsrt$rtime)
        rtmax <- c(rtmax, rtx)


        rtmd <- median(spsrt$rtime, na.rm = TRUE)
        rtmed <- c(rtmed, rtmd)

        rtmn <- mean(spsrt$rtime, na.rm = TRUE)
        rtmean <- c(rtmean, rtmn)

        #collision energy
        ce <- max(spsrt$collisionEnergy)
        col_eng <- c(col_eng, ce)

        #polarity
        pl <- max(spsrt$polarity)
        if (pl == 1){
            px <- 'pos'
            pol <- c(pol, px)
        }
        else {
            px <- 'neg'
            pol <- c(pol, px)
        }

        #int 
        ints <- max(spsrt$precursorIntensity)
        int <- c(int, ints) 

        source_file <- c(source_file, mzml_file)

        # after all the common infromation is stored, 
        # move to extracting matching candidates with input spectra

        #### input spec with pre_mz
        sps <- spec2_Processing(x, sps_all, spec = "spec_all")


        # if the database selected is GNPS or all
        if (db == "all" || db =="gnps"){

            #### GNPS spec with pre_mz
            gnps_with_mz <- spec2_Processing(x, gnpsdb, spec = "gnps", ppmx) # change here later

            dir_name <- paste(input_dir, str_remove(paste(result_dir, "/spectral_dereplication/GNPS/", sep = ""), "."), sep ="")
            if (!file.exists(dir_name)){
                dir.create(dir_name, recursive = TRUE)
            }
            
            #' Compare experimental spectra against GNPS
            res <- compareSpectra(sps, gnps_with_mz, ppm = 15, FUN = MsCoreUtils::gnps, MAPFUN = joinPeaksGnps)

            if (length(sps) > 1 && length(gnps_with_mz) >1){
                
                # given threshold of 0.90 for GNPS, extract top candidates
                res_top <- which(res > res[res>0.90], arr.ind = TRUE)
                res_topdf <- data.frame(res_top)

                # to store the scores to add to res_topdf
                gnps_scores <- c()
                # for all rows and columns in res_topdf
                for (i in 1:nrow(res_topdf)){
                    # store the scores
                    gnps_scores <- c(gnps_scores, res[(res_topdf[i, "row"]), (res_topdf[i, "col"])])
                }
                # add the score column to res_top
                gnps_res <- cbind(res_top, gnps_scores)
                # sort in descending order
                ordered_gnps_res <- gnps_res[order(-gnps_res[,"gnps_scores"]),]
                df_ord_gnps_res <- data.frame(ordered_gnps_res)
                GNPSmax_similarity <- c()
                GNPSmzScore <- c()
                GNPSintScore <- c()
                GQMatchingPeaks <- c()
                GNPSTotalPeaks <- c()
                gQueryTotalPeaks<- c()
                GNPSSMILES <- c()
                GNPSspectrumID <- c()
                GNPScompound_name <- c()
                GNPSmirrorSpec <- c()
                Source <- c()

                for (k in 1:nrow(df_ord_gnps_res)){
                    idv <- df_ord_gnps_res[k,]
                    df_peaklists <- peakdf(gnps_with_mz[idv[[2]]], sps[idv[[1]]], ppmx)
                    if (!(is.null(df_peaklists))){
                        dir_premz <- paste(input_dir, str_remove(paste(result_dir, "/spectral_dereplication/GNPS/", sep = ""), "."),as.character(id_Xx), sep ="")
                        if (!file.exists(dir_premz)){
                            dir.create(dir_premz, recursive = TRUE)
                        }

                        GNPSscore <- idv[[3]]
                        GNPSmax_similarity <- c(GNPSmax_similarity, GNPSscore)

                        GNPSmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(gnps_with_mz[idv[[2]]])[[1]])+nrow(peaksData(sps[idv[[1]]])[[1]]))
                        GNPSmzScore <- c(GNPSmzScore, GNPSmz)

                        GNPSint <- mean(1-(df_peaklists[,"diff"]/100))
                        GNPSintScore <- c(GNPSintScore, GNPSint)

                        GQMatPeaks <- nrow(df_peaklists)
                        GQMatchingPeaks <- c(GQMatchingPeaks, GQMatPeaks)

                        GNPSTPeaks <- nrow(peaksData(gnps_with_mz[idv[[2]]])[[1]])
                        GNPSTotalPeaks <- c(GNPSTotalPeaks, GNPSTPeaks)

                        gQTPeaks<- nrow(peaksData(sps[idv[[1]]])[[1]])
                        gQueryTotalPeaks <- c(gQueryTotalPeaks, gQTPeaks)

                        GNPS_SMILES <- gnps_with_mz[idv[[2]]]$SMILES
                        GNPSSMILES <- c(GNPSSMILES, GNPS_SMILES)

                        GNPSID <- gnps_with_mz[idv[[2]]]$SPECTRUMID
                        GNPSspectrumID <- c(GNPSspectrumID, GNPSID)

                        GNPSname <- gnps_with_mz[idv[[2]]]$NAME
                        GNPScompound_name <- c(GNPScompound_name, GNPSname)

                        #' plotMirror
                        name_plotmirror <- paste(dir_premz, id_Xx,"_spectra_vs_", gnps_with_mz[idv[[2]]]$SPECTRUMID, "_spectra.pdf", sep ="")
                        pdf(name_plotmirror)
                        plotSpectraMirror(sps[idv[[1]]], gnps_with_mz[idv[[2]]], tolerance = 0.2,
                                            labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                            labelSrt = -30)
                        grid()
                        dev.off()

                        GNPSSpec <- str_replace(name_plotmirror, input_dir, "./")
                        GNPSmirrorSpec <- c(GNPSmirrorSpec, GNPSSpec)

                        Src <- "GNPS"
                        Source <- c(Source, Src)

                    }
                    else{
                        GNPSscore <- NA
                        GNPSmax_similarity <- c(GNPSmax_similarity, GNPSscore)

                        GNPSmz <- NA
                        GNPSmzScore <- c(GNPSmzScore, GNPSmz)


                        GNPSint <- NA
                        GNPSintScore <- c(GNPSintScore, GNPSint)


                        GQMatPeaks <- NA
                        GQMatchingPeaks <- c(GQMatchingPeaks, GQMatPeaks)


                        GNPSTPeaks <- NA
                        GNPSTotalPeaks <- c(GNPSTotalPeaks, GNPSTPeaks)


                        gQTPeaks<- NA
                        gQueryTotalPeaks <- c(gQueryTotalPeaks, gQTPeaks)


                        GNPS_SMILES <- NA
                        GNPSSMILES <- c(GNPSSMILES, GNPS_SMILES)


                        GNPSID <- NA
                        GNPSspectrumID <- c(GNPSspectrumID, GNPSID)


                        GNPSname <- NA
                        GNPScompound_name <- c(GNPScompound_name, GNPSname)


                        GNPSSpec <- NA
                        GNPSmirrorSpec <- c(GNPSmirrorSpec, GNPSSpec)


                        Src <- NA
                        Source <- c(Source, Src)
                    }
                }

                gnps_x <- data.frame(cbind(GNPSmax_similarity, GNPSmzScore, 
                                           GNPSintScore, GQMatchingPeaks, 
                                           GNPSTotalPeaks, gQueryTotalPeaks, 
                                           GNPSSMILES, GNPSspectrumID, GNPScompound_name, 
                                           GNPSmirrorSpec, Source))
                write.csv(gnps_x, file = paste(dir_premz, "/gnps_results_for_", id_Xx, ".csv", sep = ""))
            }
            else if (length(sps) == 1 && length(gnps_with_mz) >1){
                
                # given threshold of 0.90 for GNPS, extract top candidates
                res_top <- which(res > res[res>0.90], arr.ind = TRUE)
                res_topdf <- data.frame(res_top)

                # top store the scores to add to res_topdf
                gnps_scores <- c()
                # for all rows and columns in res_topdf
                
                for (i in 1:nrow(res_topdf)){
                    # store the scores
                    gnps_scores <- c(gnps_scores, res[(res_topdf[i, "row"]), (res_topdf[i, "col"])])
                }
                
                # add the score column to res_top
                gnps_res <- cbind(res_top, gnps_scores)
                
                # sort in descending order
                ordered_gnps_res <- gnps_res[order(-gnps_res[,"gnps_scores"]),]
                
                df_ord_gnps_res <- data.frame(ordered_gnps_res)
                
                GNPSmax_similarity <- c()
                GNPSmzScore <- c()
                GNPSintScore <- c()
                GQMatchingPeaks <- c()
                GNPSTotalPeaks <- c()
                gQueryTotalPeaks<- c()
                GNPSSMILES <- c()
                GNPSspectrumID <- c()
                GNPScompound_name <- c()
                GNPSmirrorSpec <- c()
                Source <- c()
                
                for (k in 1:nrow(df_ord_gnps_res)){
                    idv <- df_ord_gnps_res[k,]
                    df_peaklists <- peakdf(gnps_with_mz[idv[[2]]], sps, ppmx)
                    if (!(is.null(df_peaklists))){
                        dir_premz <- paste(input_dir, str_remove(paste(result_dir, "/spectral_dereplication/GNPS/", sep = ""), "."),as.character(id_Xx), sep ="")
                        if (!file.exists(dir_premz)){
                            dir.create(dir_premz, recursive = TRUE)
                        }

                        GNPSscore <- idv[[3]]
                        GNPSmax_similarity <- c(GNPSmax_similarity, GNPSscore)

                        GNPSmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(gnps_with_mz[idv[[2]]])[[1]])+nrow(peaksData(sps)[[1]]))
                        GNPSmzScore <- c(GNPSmzScore, GNPSmz)

                        GNPSint <- mean(1-(df_peaklists[,"diff"]/100))
                        GNPSintScore <- c(GNPSintScore, GNPSint)

                        GQMatPeaks <- nrow(df_peaklists)
                        GQMatchingPeaks <- c(GQMatchingPeaks, GQMatPeaks)

                        GNPSTPeaks <- nrow(peaksData(gnps_with_mz[idv[[2]]])[[1]])
                        GNPSTotalPeaks <- c(GNPSTotalPeaks, GNPSTPeaks)

                        gQTPeaks<- nrow(peaksData(sps)[[1]])
                        gQueryTotalPeaks <- c(gQueryTotalPeaks, gQTPeaks)

                        GNPS_SMILES <- gnps_with_mz[idv[[2]]]$SMILES
                        GNPSSMILES <- c(GNPSSMILES, GNPS_SMILES)

                        GNPSID <- gnps_with_mz[idv[[2]]]$SPECTRUMID
                        GNPSspectrumID <- c(GNPSspectrumID, GNPSID)

                        GNPSname <- gnps_with_mz[idv[[2]]]$NAME
                        GNPScompound_name <- c(GNPScompound_name, GNPSname)

                        #' plotMirror
                        name_plotmirror <- paste(dir_premz, id_Xx,"_spectra_vs_", gnps_with_mz[idv[[2]]]$SPECTRUMID, "_spectra.pdf", sep ="")
                        pdf(name_plotmirror)
                        plotSpectraMirror(sps, gnps_with_mz[idv[[2]]], tolerance = 0.2,
                                            labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                            labelSrt = -30)
                        grid()
                        dev.off()

                        GNPSSpec <- str_replace(name_plotmirror, input_dir, "./")
                        GNPSmirrorSpec <- c(GNPSmirrorSpec, GNPSSpec)

                        Src <- "GNPS"
                        Source <- c(Source, Src)

                    }
                    else{
                        GNPSscore <- NA
                        GNPSmax_similarity <- c(GNPSmax_similarity, GNPSscore)

                        GNPSmz <- NA
                        GNPSmzScore <- c(GNPSmzScore, GNPSmz)


                        GNPSint <- NA
                        GNPSintScore <- c(GNPSintScore, GNPSint)


                        GQMatPeaks <- NA
                        GQMatchingPeaks <- c(GQMatchingPeaks, GQMatPeaks)


                        GNPSTPeaks <- NA
                        GNPSTotalPeaks <- c(GNPSTotalPeaks, GNPSTPeaks)


                        gQTPeaks<- NA
                        gQueryTotalPeaks <- c(gQueryTotalPeaks, gQTPeaks)


                        GNPS_SMILES <- NA
                        GNPSSMILES <- c(GNPSSMILES, GNPS_SMILES)


                        GNPSID <- NA
                        GNPSspectrumID <- c(GNPSspectrumID, GNPSID)


                        GNPSname <- NA
                        GNPScompound_name <- c(GNPScompound_name, GNPSname)


                        GNPSSpec <- NA
                        GNPSmirrorSpec <- c(GNPSmirrorSpec, GNPSSpec)


                        Src <- NA
                        Source <- c(Source, Src)
                    }
                }
            }else if (length(sps) > 1 && length(gnps_with_mz) == 1){
                # given threshold of 0.90 for GNPS, extract top candidates
                res_top <- which(res > res[res>0.90], arr.ind = TRUE)
                res_topdf <- data.frame(res_top)

                # top store the scores to add to res_topdf
                gnps_scores <- c()
                # for all rows and columns in res_topdf
                
                for (i in 1:nrow(res_topdf)){
                    # store the scores
                    gnps_scores <- c(gnps_scores, res[(res_topdf[i, "row"]), (res_topdf[i, "col"])])
                }
                
                # add the score column to res_top
                gnps_res <- cbind(res_top, gnps_scores)
                
                # sort in descending order
                ordered_gnps_res <- gnps_res[order(-gnps_res[,"gnps_scores"]),]
                
                df_ord_gnps_res <- data.frame(ordered_gnps_res)
                
                GNPSmax_similarity <- c()
                GNPSmzScore <- c()
                GNPSintScore <- c()
                GQMatchingPeaks <- c()
                GNPSTotalPeaks <- c()
                gQueryTotalPeaks<- c()
                GNPSSMILES <- c()
                GNPSspectrumID <- c()
                GNPScompound_name <- c()
                GNPSmirrorSpec <- c()
                Source <- c()
                for (k in 1:nrow(df_ord_gnps_res)){
                    idv <- df_ord_gnps_res[k,]
                    df_peaklists <- peakdf(gnps_with_mz[idv[[2]]], sps, ppmx)
                    if (!(is.null(df_peaklists))){
                        dir_premz <- paste(input_dir, str_remove(paste(result_dir, "/spectral_dereplication/GNPS/", sep = ""), "."),as.character(id_Xx), sep ="")
                        if (!file.exists(dir_premz)){
                            dir.create(dir_premz, recursive = TRUE)
                        }

                        GNPSscore <- idv[[3]]
                        GNPSmax_similarity <- c(GNPSmax_similarity, GNPSscore)

                        GNPSmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(gnps_with_mz[idv[[2]]])[[1]])+nrow(peaksData(sps)[[1]]))
                        GNPSmzScore <- c(GNPSmzScore, GNPSmz)

                        GNPSint <- mean(1-(df_peaklists[,"diff"]/100))
                        GNPSintScore <- c(GNPSintScore, GNPSint)

                        GQMatPeaks <- nrow(df_peaklists)
                        GQMatchingPeaks <- c(GQMatchingPeaks, GQMatPeaks)

                        GNPSTPeaks <- nrow(peaksData(gnps_with_mz[idv[[2]]])[[1]])
                        GNPSTotalPeaks <- c(GNPSTotalPeaks, GNPSTPeaks)

                        gQTPeaks<- nrow(peaksData(sps)[[1]])
                        gQueryTotalPeaks <- c(gQueryTotalPeaks, gQTPeaks)

                        GNPS_SMILES <- gnps_with_mz[idv[[2]]]$SMILES
                        GNPSSMILES <- c(GNPSSMILES, GNPS_SMILES)

                        GNPSID <- gnps_with_mz[idv[[2]]]$SPECTRUMID
                        GNPSspectrumID <- c(GNPSspectrumID, GNPSID)

                        GNPSname <- gnps_with_mz[idv[[2]]]$NAME
                        GNPScompound_name <- c(GNPScompound_name, GNPSname)

                        #' plotMirror
                        name_plotmirror <- paste(dir_premz, id_Xx,"_spectra_vs_", gnps_with_mz[idv[[2]]]$SPECTRUMID, "_spectra.pdf", sep ="")
                        pdf(name_plotmirror)
                        plotSpectraMirror(sps, gnps_with_mz[idv[[2]]], tolerance = 0.2,
                                            labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                            labelSrt = -30)
                        grid()
                        dev.off()

                        GNPSSpec <- str_replace(name_plotmirror, input_dir, "./")
                        GNPSmirrorSpec <- c(GNPSmirrorSpec, GNPSSpec)

                        Src <- "GNPS"
                        Source <- c(Source, Src)

                    }else{
                        GNPSscore <- NA
                        GNPSmax_similarity <- c(GNPSmax_similarity, GNPSscore)

                        GNPSmz <- NA
                        GNPSmzScore <- c(GNPSmzScore, GNPSmz)


                        GNPSint <- NA
                        GNPSintScore <- c(GNPSintScore, GNPSint)


                        GQMatPeaks <- NA
                        GQMatchingPeaks <- c(GQMatchingPeaks, GQMatPeaks)


                        GNPSTPeaks <- NA
                        GNPSTotalPeaks <- c(GNPSTotalPeaks, GNPSTPeaks)


                        gQTPeaks<- NA
                        gQueryTotalPeaks <- c(gQueryTotalPeaks, gQTPeaks)


                        GNPS_SMILES <- NA
                        GNPSSMILES <- c(GNPSSMILES, GNPS_SMILES)


                        GNPSID <- NA
                        GNPSspectrumID <- c(GNPSspectrumID, GNPSID)


                        GNPSname <- NA
                        GNPScompound_name <- c(GNPScompound_name, GNPSname)


                        GNPSSpec <- NA
                        GNPSmirrorSpec <- c(GNPSmirrorSpec, GNPSSpec)


                        Src <- NA
                        Source <- c(Source, Src)
                    }
                    
                }
                
            }
            
            else if (length(sps) == 1 && length(gnps_with_mz) == 1){
                gnps_best_match <- gnps_with_mz
                df_peaklists <- peakdf(gnps_best_match, sps, ppmx)
                if (!(is.null(df_peaklists))){
                    dir_premz <- paste(input_dir, str_remove(paste(result_dir, "/spectral_dereplication/GNPS/", sep = ""), "."),as.character(id_Xx), sep ="")
                    if (!file.exists(dir_premz)){
                        dir.create(dir_premz, recursive = TRUE)
                    }
                    #' plotMirror
                    name_plotmirror <- paste(dir_name, x,"_spectra_vs_", gnps_best_match$SPECTRUMID, "_spectra.pdf", sep ="")
                    pdf(name_plotmirror)
                    plotSpectraMirror(sps, gnps_best_match, tolerance = 0.2,
                                        labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                        labelSrt = -30)
                    grid()
                    dev.off()


                    GNPSscore <- max(res)
                    GNPSmax_similarity <- c(GNPSmax_similarity, GNPSscore)
                
                
                
                    GNPSmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(gnps_best_match)[[1]])+nrow(peaksData(sps)[[1]]))
                    GNPSmzScore <- c(GNPSmzScore, GNPSmz)

                    GNPSint <- mean(1-(df_peaklists[,"diff"]/100))
                    GNPSintScore <- c(GNPSintScore, GNPSint)

                    GQMatPeaks <- NA
                    GQMatchingPeaks <- c(GQMatchingPeaks, GQMatPeaks)


                    GNPSTPeaks <- nrow(peaksData(gnps_best_match)[[1]])
                    GNPSTotalPeaks <- c(GNPSTotalPeaks, GNPSTPeaks)                

                    gQTPeaks<- nrow(peaksData(sps)[[1]])
                    gQueryTotalPeaks <- c(gQueryTotalPeaks, gQTPeaks)


                    GNPS_SMILES <- gnps_best_match$SMILES
                    GNPSSMILES <- c(GNPSSMILES, GNPS_SMILES)

                    GNPSname <- gnps_best_match$NAME
                    GNPScompound_name <- c(GNPScompound_name, GNPSname)


                    GNPSID <- gnps_best_match$SPECTRUMID
                    GNPSspectrumID <- c(GNPSspectrumID, GNPSID)


                    GNPSSpec <- NA
                    GNPSmirrorSpec <- c(GNPSmirrorSpec, GNPSSpec)


                    Src <- "GNPS"
                    Source <- c(Source, Src)
                }
                else{


                    GNPSscore <- NA
                    GNPSmax_similarity <- c(GNPSmax_similarity, GNPSscore)


                    GNPSmz <- NA
                    GNPSmzScore <- c(GNPSmzScore, GNPSmz)


                    GNPSint <- NA
                    GNPSintScore <- c(GNPSintScore, GNPSint)


                    GQMatPeaks <- NA
                    GQMatchingPeaks <- c(GQMatchingPeaks, GQMatPeaks)


                    GNPSTPeaks <- NA
                    GNPSTotalPeaks <- c(GNPSTotalPeaks, GNPSTPeaks)


                    gQTPeaks<- NA
                    gQueryTotalPeaks <- c(gQueryTotalPeaks, gQTPeaks)


                    GNPS_SMILES <- NA
                    GNPSSMILES <- c(GNPSSMILES, GNPS_SMILES)


                    GNPSID <- NA
                    GNPSspectrumID <- c(GNPSspectrumID, GNPSID)


                    GNPSname <- NA
                    GNPScompound_name <- c(GNPScompound_name, GNPSname)


                    GNPSSpec <- NA
                    GNPSmirrorSpec <- c(GNPSmirrorSpec, GNPSSpec)


                    Src <- NA
                    Source <- c(Source, Src)
                }
            
            } # ends 4th condition gnps
           
        } # ends gnps
            
    } # ends each pre mz
    result_dir_spectra <- paste(input_dir, str_remove(paste(result_dir, "/spectral_dereplication", sep = ""), "."), sep = "")
    spectra_input <- data.frame(cbind(id_X, premz, rtmin, 
                                      rtmax, rtmed, rtmean, 
                                      col_eng, pol, int, source_file))
    write.csv(spectra_input, file = paste(result_dir_spectra, "/gnps_results_for_", file_id, ".csv", sep = ""))
}

In [ ]:



                    #' plotMirror
                    name_plotmirror <- paste(dir_name, x,"_spectra_vs_", hmdb_best_match$compound_id, "_spectra.pdf", sep ="")
                    pdf(name_plotmirror)
                    plotSpectraMirror(sps[idx[1]], hmdb_with_mz[idx[2]], tolerance = 0.2,
                                        labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                        labelSrt = -30)
                    grid()
                    dev.off()


                    HMDBscore <- max(res)
                    HMDBmax_similarity <- c(HMDBmax_similarity, HMDBscore)

                    HMDBmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(hmdb_best_match)[[1]])+nrow(peaksData(sps[idx[1]])[[1]]))
                    HMDBmzScore <- c(HMDBmzScore, HMDBmz)

                    HMDBint <- mean(1-(df_peaklists[,"diff"]/100))
                    HMDBintScore <- c(HMDBintScore, HMDBint)


                    HQMatPeaks <- nrow(df_peaklists)
                    HQMatchingPeaks <- c(HQMatchingPeaks, HQMatPeaks)


                    HMDBTPeaks <- nrow(peaksData(hmdb_best_match)[[1]])
                    HMDBTotalPeaks <- c(HMDBTotalPeaks, HMDBTPeaks)

                    hQTPeaks<- nrow(peaksData(sps[idx[1]])[[1]])
                    hQueryTotalPeaks<- c(hQueryTotalPeaks, hQTPeaks)


                    HMDBID <- hmdb_best_match$compound_id
                    HMDBcompoundID <- c(HMDBcompoundID, HMDBID)

                    HMDBSpec <- str_replace(name_plotmirror, input_dir, "./")
                    HMDBmirrorSpec <- c(HMDBmirrorSpec, HMDBSpec)

                    Src <- "HMDB"
                    Source <- c(Source, Src)
                }
                else{

                    HMDBscore <- NA
                    HMDBmax_similarity <- c(HMDBmax_similarity, HMDBscore)


                    HMDBmz <- NA
                    HMDBmzScore <- c(HMDBmzScore, HMDBmz)


                    HMDBint <- NA
                    HMDBintScore <- c(HMDBintScore, HMDBint)


                    HQMatPeaks <- NA
                    HQMatchingPeaks <- c(HQMatchingPeaks, HQMatPeaks)


                    HMDBTPeaks <- NA
                    HMDBTotalPeaks <- c(HMDBTotalPeaks, HMDBTPeaks)


                    hQTPeaks<- NA
                    hQueryTotalPeaks<- c(hQueryTotalPeaks, hQTPeaks)


                    HMDBID <- "none"
                    HMDBcompoundID <- c(HMDBcompoundID, HMDBID)


                    HMDBSpec <- NA
                    HMDBmirrorSpec <- c(HMDBmirrorSpec, HMDBSpec)


                    Src <- NA
                    Source <- c(Source, Src)
                }
            }
            else if (length(sps) == 1 && length(hmdb_with_mz) >1){
                #' Compare experimental spectra against HMDB
                res <- compareSpectra(sps, hmdb_with_mz, ppm = 15)

                #' obtain HMDB spectra that matches the most with m/z MS2 spectra
                gx <- which(res == max(res))
                gx <- gx[1]
                hmdb_best_match <- hmdb_with_mz[gx]

                df_peaklists <- peakdf(hmdb_best_match, sps, ppmx)


                #' if there are more than 2 peak matching
                if (!(is.null(df_peaklists))){

                    #' plotMirror
                    name_plotmirror <- paste(dir_name, x,"_spectra_vs_", hmdb_best_match$compound_id, "_spectra.pdf", sep ="")
                    pdf(name_plotmirror)
                    plotSpectraMirror(sps, hmdb_best_match, tolerance = 0.2,
                                        labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                        labelSrt = -30)
                    grid()
                    dev.off() 

                    HMDBscore <- max(res)
                    HMDBmax_similarity <- c(HMDBmax_similarity, HMDBscore)


                    HMDBmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(hmdb_best_match)[[1]])+nrow(peaksData(sps)[[1]]))
                    HMDBmzScore <- c(HMDBmzScore, HMDBmz)

                    HMDBint <- mean(1-(df_peaklists[,"diff"]/100))
                    HMDBintScore <- c(HMDBintScore, HMDBint)


                    HQMatPeaks <- nrow(df_peaklists)
                    HQMatchingPeaks <- c(HQMatchingPeaks, HQMatPeaks)


                    HMDBTPeaks <- nrow(peaksData(hmdb_best_match)[[1]])
                    HMDBTotalPeaks <- c(HMDBTotalPeaks, HMDBTPeaks)


                    hQTPeaks<- nrow(peaksData(sps)[[1]])
                    hQueryTotalPeaks<- c(hQueryTotalPeaks, hQTPeaks)


                    HMDBID <- hmdb_best_match$compound_id
                    HMDBcompoundID <- c(HMDBcompoundID, HMDBID)



                    HMDBSpec <- str_replace(name_plotmirror, input_dir, "./")
                    HMDBmirrorSpec <- c(HMDBmirrorSpec, HMDBSpec)


                    Src <- "HMDB"
                    Source <- c(Source, Src)
                }
                else{

                    HMDBscore <- NA
                    HMDBmax_similarity <- c(HMDBmax_similarity, HMDBscore)


                    HMDBmz <- NA
                    HMDBmzScore <- c(HMDBmzScore, HMDBmz)


                    HMDBint <- NA
                    HMDBintScore <- c(HMDBintScore, HMDBint)


                    HQMatPeaks <- NA
                    HQMatchingPeaks <- c(HQMatchingPeaks, HQMatPeaks)


                    HMDBTPeaks <- NA
                    HMDBTotalPeaks <- c(HMDBTotalPeaks, HMDBTPeaks)


                    hQTPeaks<- NA
                    hQueryTotalPeaks<- c(hQueryTotalPeaks, hQTPeaks)


                    HMDBID <- "none"
                    HMDBcompoundID <- c(HMDBcompoundID, HMDBID)


                    HMDBSpec <- NA
                    HMDBmirrorSpec <- c(HMDBmirrorSpec, HMDBSpec)


                    Src <- NA
                    Source <- c(Source, Src)
                }

            }
            else if (length(sps) > 1 && length(hmdb_with_mz) == 1){
                #' Compare experimental spectra against HMDB
                res <- compareSpectra(sps, hmdb_with_mz, ppm = 15)
                #' obtain hmdb spectra that matches the most with m/z MS2 spectra

                gx <- which(res == max(res))
                gx <- gx[1]
                sps <- sps[gx]
                df_peaklists <- peakdf(hmdb_with_mz, sps[gx], ppmx)
                hmdb_best_match <- hmdb_with_mz
                #' if there are more than 2 peak matching
                if (!(is.null(df_peaklists))){


                    #' plotMirror
                    name_plotmirror <- paste(dir_name, x,"_spectra_vs_", hmdb_best_match$compound_id, "_spectra.pdf", sep ="")
                    pdf(name_plotmirror)
                    plotSpectraMirror(sps, hmdb_best_match, tolerance = 0.2,
                                        labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                        labelSrt = -30)
                    grid()
                    dev.off()

                    HMDBscore <- max(res)
                    HMDBmax_similarity <- c(HMDBmax_similarity, HMDBscore)


                    HMDBmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(hmdb_best_match)[[1]])+nrow(peaksData(sps)[[1]]))
                    HMDBmzScore <- c(HMDBmzScore, HMDBmz)

                    HMDBint <- mean(1-(df_peaklists[,"diff"]/100))
                    HMDBintScore <- c(HMDBintScore, HMDBint)


                    HQMatPeaks <- nrow(df_peaklists)
                    HQMatchingPeaks <- c(HQMatchingPeaks, HQMatPeaks)


                    HMDBTPeaks <- nrow(peaksData(hmdb_best_match)[[1]])
                    HMDBTotalPeaks <- c(HMDBTotalPeaks, HMDBTPeaks)


                    hQTPeaks<- nrow(peaksData(sps)[[1]])
                    hQueryTotalPeaks<- c(hQueryTotalPeaks, hQTPeaks)


                    HMDBID <- hmdb_best_match$compound_id
                    HMDBcompoundID <- c(HMDBcompoundID, HMDBID)



                    HMDBSpec <- str_replace(name_plotmirror, input_dir, "./")
                    HMDBmirrorSpec <- c(HMDBmirrorSpec, HMDBSpec)


                    Src <- "HMDB"
                    Source <- c(Source, Src)
                    }
                else{

                    HMDBscore <- NA
                    HMDBmax_similarity <- c(HMDBmax_similarity, HMDBscore)


                    HMDBmz <- NA
                    HMDBmzScore <- c(HMDBmzScore, HMDBmz)


                    HMDBint <- NA
                    HMDBintScore <- c(HMDBintScore, HMDBint)



                    HQMatPeaks <- NA
                    HQMatchingPeaks <- c(HQMatchingPeaks, HQMatPeaks)


                    HMDBTPeaks <- NA
                    HMDBTotalPeaks <- c(HMDBTotalPeaks, HMDBTPeaks)


                    hQTPeaks<- NA
                    hQueryTotalPeaks<- c(hQueryTotalPeaks, hQTPeaks)


                    HMDBID <- "none"
                    HMDBcompoundID <- c(HMDBcompoundID, HMDBID)


                    HMDBSpec <- NA
                    HMDBmirrorSpec <- c(HMDBmirrorSpec, HMDBSpec)


                    Src <- NA
                    Source <- c(Source, Src)
                }
            }
            else if (length(sps) == 1 && length(hmdb_with_mz) == 1){
                #' Compare experimental spectra against HMDB
                res <- compareSpectra(sps, hmdb_with_mz, ppm = 15)
                hmdb_best_match <- hmdb_with_mz
                df_peaklists <- peakdf(hmdb_best_match, sps, ppmx)
                if (!(is.null(df_peaklists))){
                    #' plotMirror
                    name_plotmirror <- paste(dir_name, x,"_spectra_vs_", hmdb_best_match$compound_id, "_spectra.pdf", sep ="")
                    pdf(name_plotmirror)
                    plotSpectraMirror(sps, hmdb_best_match, tolerance = 0.2,
                                        labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                        labelSrt = -30)
                    grid()
                    dev.off()

                    HMDBscore <- max(res)
                    HMDBmax_similarity <- c(HMDBmax_similarity, HMDBscore)


                    HMDBmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(hmdb_best_match)[[1]])+nrow(peaksData(sps)[[1]]))
                    HMDBmzScore <- c(HMDBmzScore, HMDBmz)


                    HMDBint <- mean(1-(df_peaklists[,"diff"]/100))
                    HMDBintScore <- c(HMDBintScore, HMDBint)



                    HQMatPeaks <- nrow(df_peaklists)
                    HQMatchingPeaks <- c(HQMatchingPeaks, HQMatPeaks)


                    HMDBTPeaks <- nrow(peaksData(hmdb_best_match)[[1]])
                    HMDBTotalPeaks <- c(HMDBTotalPeaks, HMDBTPeaks)


                    hQTPeaks<- nrow(peaksData(sps)[[1]])
                    hQueryTotalPeaks<- c(hQueryTotalPeaks, hQTPeaks)


                    HMDBID <- hmdb_best_match$compound_id
                    HMDBcompoundID <- c(HMDBcompoundID, HMDBID)



                    HMDBSpec <- str_replace(name_plotmirror, input_dir, "./")
                    HMDBmirrorSpec <- c(HMDBmirrorSpec, HMDBSpec)


                    Src <- "HMDB"
                    Source <- c(Source, Src)
                }
                else{

                    HMDBscore <- NA
                    HMDBmax_similarity <- c(HMDBmax_similarity, HMDBscore)


                    HMDBmz <- NA
                    HMDBmzScore <- c(HMDBmzScore, HMDBmz)


                    HMDBint <- NA
                    HMDBintScore <- c(HMDBintScore, HMDBint)


                    HQMatPeaks <- NA
                    HQMatchingPeaks <- c(HQMatchingPeaks, HQMatPeaks)


                    HMDBTPeaks <- NA
                    HMDBTotalPeaks <- c(HMDBTotalPeaks, HMDBTPeaks)


                    hQTPeaks<- NA
                    hQueryTotalPeaks<- c(hQueryTotalPeaks, hQTPeaks)


                    HMDBID <- "none"
                    HMDBcompoundID <- c(HMDBcompoundID, HMDBID)


                    HMDBSpec <- NA
                    HMDBmirrorSpec <- c(HMDBmirrorSpec, HMDBSpec)


                    Src <- NA
                    Source <- c(Source, Src)
                }

            }
            else{


                HMDBscore <- NA
                HMDBmax_similarity <- c(HMDBmax_similarity, HMDBscore)


                HMDBmz <- NA
                HMDBmzScore <- c(HMDBmzScore, HMDBmz)


                HMDBint <- NA
                HMDBintScore <- c(HMDBintScore, HMDBint)


                HQMatPeaks <- NA
                HQMatchingPeaks <- c(HQMatchingPeaks, HQMatPeaks)


                HMDBTPeaks <- NA
                HMDBTotalPeaks <- c(HMDBTotalPeaks, HMDBTPeaks)


                hQTPeaks<- NA
                hQueryTotalPeaks<- c(hQueryTotalPeaks, hQTPeaks)


                HMDBID <- "none"
                HMDBcompoundID <- c(HMDBcompoundID, HMDBID)

                HMDBSpec <- NA
                HMDBmirrorSpec <- c(HMDBmirrorSpec, HMDBSpec)


                Src <- NA
                Source <- c(Source, Src)
            }
        }
        df_hmdb <- cbind(id_X, premz, rtmin, rtmax, rtmed, rtmean, HMDBmax_similarity, HMDBmzScore, 
                         HMDBintScore, HQMatchingPeaks, HMDBTotalPeaks, hQueryTotalPeaks, 
                         HMDBcompoundID, HMDBmirrorSpec, Source)

        #write.csv(df_hmdb, str_remove(paste(result_dir, "/spectral_dereplication/hmdb.csv", sep = ""), "./"))
        write.csv(df_hmdb, paste(input_dir, str_remove(paste(result_dir, "/spectral_dereplication/hmdb.csv", sep = ""), "./"), sep = ""))
    }
    ####-------------------------------------------------------------
    #### Dereplication with all or MassBank ----
     ####-------------------------------------------------------------
    if (db == "all" || db =="mbank"){

        load(file = paste(input_dir,"mbankNIST.rda", sep = ""))
        
        # common

        id_X <- c()
        premz <- c()
        rtmin <- c()
        rtmax <- c()
        rtmed <- c()
        rtmean <- c()

        # mbank
        MBmax_similarity <- c()
        MBmzScore <- c()
        MBintScore <- c()
        MQMatchingPeaks <- c()
        MBTotalPeaks <- c()
        mQueryTotalPeaks<- c()
        MBformula <- c()
        MBinchiKEY <- c()
        MBspectrumID <- c()
        MBcompound_name <- c()
        MBmirrorSpec <- c()

        # common
        Source <- c()
        nx <- 0
        for (x in pre_mz){

            
            nx <- nx+1
            
            
            spsrt <- filterPrecursorMzRange(sps_all, x)
        
            
            id_Xx <- paste(file_id,  "M",  as.character(round(x, digits = 0)), 
                            "R", as.character(round(median(spsrt$rtime, na.rm = TRUE), digits = 0)), 
                            "ID", as.character(nx), sep = '')
            id_X <- c(id_X, id_Xx)

            pre <- x
            premz <- c(premz, pre)

            rti <- min(spsrt$rtime)
            rtmin <- c(rtmin, rti)

            rtx <- max(spsrt$rtime)
            rtmax <- c(rtmax, rtx)


            rtmd <- median(spsrt$rtime, na.rm = TRUE)
            rtmed <- c(rtmed, rtmd)

            rtmn <- mean(spsrt$rtime, na.rm = TRUE)
            rtmean <- c(rtmean, rtmn)

            #### input spec with pre_mz
            sps <- spec2_Processing(x, sps_all, spec = "spec_all")

            #### GNPS spec with pre_mz
            mbank_with_mz <- spec2_Processing(x, mbank, spec = "mbank", ppmx = 15)
            
            

            dir_name <- paste(input_dir, str_remove(paste(result_dir, "/spectral_dereplication/MassBank/", sep = ""), "./"), sep ="")
            if (!file.exists(dir_name)){
                dir.create(dir_name, recursive = TRUE)
            }
            if (length(sps) > 1 && length(mbank_with_mz) >1){

                #' Compare experimental spectra against MassBank
                res <- compareSpectra(sps, mbank_with_mz, ppm = 15)

                #' obtain GNPS spectra that matches the most with m/z MS2 spectra
                idx <- which(res == max(res), arr.ind = TRUE)
                
                if (nrow(idx)>1){
                    idx <- idx[1,]
                }
                
                
                mbank_best_match <- mbank_with_mz[idx[2]]
                df_peaklists <- peakdf(mbank_best_match, sps[idx[1]], ppmx)

                if (!(is.null(df_peaklists))){

                    #' plotMirror
                    name_plotmirror <- paste(dir_name, x,"_spectra_vs_", mbank_best_match$accession, "_spectra.pdf", sep ="")
                    pdf(name_plotmirror)
                    plotSpectraMirror(sps[idx[1]], mbank_with_mz[idx[2]], tolerance = 0.2,
                                        labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                        labelSrt = -30)
                    grid()
                    dev.off()

                    mbscore <- max(res)
                    MBmax_similarity<- c(MBmax_similarity, mbscore)

                    MBmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(mbank_best_match)[[1]])+nrow(peaksData(sps[idx[1]])[[1]]))
                    MBmzScore <- c(MBmzScore, MBmz)

                    MBint <- mean(1-(df_peaklists[,"diff"]/100))
                    MBintScore <- c(MBintScore, MBint)

                    MQMatPeaks <- nrow(df_peaklists)
                    MQMatchingPeaks <- c(MQMatchingPeaks, MQMatPeaks) 

                    MBTPeaks <- nrow(peaksData(mbank_best_match)[[1]])
                    MBTotalPeaks<- c(MBTotalPeaks, MBTPeaks)

                    mQTPeaks<- nrow(peaksData(sps[idx[1]])[[1]]) 
                    mQueryTotalPeaks<- c(mQueryTotalPeaks, mQTPeaks)


                    MBfor <- mbank_best_match$formula
                    MBformula<- c(MBformula, MBfor)

                    MBinchiK <- mbank_best_match$inchikey
                    MBinchiKEY <- c(MBinchiKEY, MBinchiK)

                    MBID <- mbank_best_match$accession
                    MBspectrumID<- c(MBspectrumID, MBID)

                    MBname <- mbank_best_match$name
                    MBcompound_name <- c(MBcompound_name, MBname)


                    MBSpec <- str_replace(name_plotmirror, input_dir, "./")
                    MBmirrorSpec <- c(MBmirrorSpec, MBSpec)

                    Src <- "MassBank"
                    Source <- c(Source, Src)
                }
                else{

                    mbscore <- NA
                    MBmax_similarity<- c(MBmax_similarity, mbscore)

                    MBmz <- NA
                    MBmzScore <- c(MBmzScore, MBmz)

                    MBint <- NA
                    MBintScore <- c(MBintScore, MBint)

                    MQMatPeaks <- NA
                    MQMatchingPeaks <- c(MQMatchingPeaks, MQMatPeaks)

                    MBTPeaks <- NA
                    MBTotalPeaks<- c(MBTotalPeaks, MBTPeaks)

                    mQTPeaks<- NA 
                    mQueryTotalPeaks<- c(mQueryTotalPeaks, mQTPeaks)

                    MBfor <- NA
                    MBformula<- c(MBformula, MBfor)

                    MBinchiK <- NA
                    MBinchiKEY <- c(MBinchiKEY, MBinchiK)

                    MBID <- NA
                    MBspectrumID<- c(MBspectrumID, MBID)

                    MBname <- NA
                    MBcompound_name <- c(MBcompound_name, MBname)

                    MBSpec <- NA
                    MBmirrorSpec <- c(MBmirrorSpec, MBSpec)

                    Src <- NA
                    Source <- c(Source, Src)
                }
            }
            else if (length(sps) == 1 && length(mbank_with_mz) >1){

                #' Compare experimental spectra against MassBank
                res <- compareSpectra(sps, mbank_with_mz, ppm = 15)
                #' obtain MassBank spectra that matches the most with m/z MS2 spectra
                gx <- which(res == max(res))
                gx <- gx[1]
                mbank_best_match <- mbank_with_mz[gx]

                df_peaklists <- peakdf(mbank_best_match, sps, ppmx)


                #' if there are more than 2 peak matching
                if (!(is.null(df_peaklists))){


                    #' plotMirror
                    name_plotmirror <- paste(dir_name, x,"_spectra_vs_", mbank_best_match$accession, "_spectra.pdf", sep ="")
                    pdf(name_plotmirror)
                    plotSpectraMirror(sps, mbank_best_match, tolerance = 0.2,
                                        labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                        labelSrt = -30)
                    grid()
                    dev.off() 

                    mbscore <- max(res)
                    MBmax_similarity<- c(MBmax_similarity, mbscore)

                    MBmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(mbank_best_match)[[1]])+nrow(peaksData(sps)[[1]]))
                    MBmzScore <- c(MBmzScore, MBmz)

                    MBint <- mean(1-(df_peaklists[,"diff"]/100))
                    MBintScore <- c(MBintScore, MBint)

                    MQMatPeaks <- nrow(df_peaklists)
                    MQMatchingPeaks <- c(MQMatchingPeaks, MQMatPeaks)

                    MBTPeaks <- nrow(peaksData(mbank_best_match)[[1]])
                    MBTotalPeaks<- c(MBTotalPeaks, MBTPeaks)

                    mQTPeaks<- nrow(peaksData(sps)[[1]])
                    mQueryTotalPeaks<- c(mQueryTotalPeaks, mQTPeaks)

                    MBfor <- mbank_best_match$formula
                    MBformula<- c(MBformula, MBfor)

                    MBinchiK <- mbank_best_match$inchikey
                    MBinchiKEY <- c(MBinchiKEY, MBinchiK)

                    MBID <- mbank_best_match$accession
                    MBspectrumID<- c(MBspectrumID, MBID)

                    MBname <- mbank_best_match$name
                    MBcompound_name <- c(MBcompound_name, MBname)

                    MBSpec <- str_replace(name_plotmirror, input_dir, "./")
                    MBmirrorSpec <- c(MBmirrorSpec, MBSpec)

                    Src <- "MassBank"
                    Source <- c(Source, Src)
                }
                else{

                    mbscore <- NA
                    MBmax_similarity<- c(MBmax_similarity, mbscore)

                    MBmz <- NA
                    MBmzScore <- c(MBmzScore, MBmz)

                    MBint <- NA
                    MBintScore <- c(MBintScore, MBint)

                    MQMatPeaks <- NA
                    MQMatchingPeaks <- c(MQMatchingPeaks, MQMatPeaks)

                    MBTPeaks <- NA
                    MBTotalPeaks<- c(MBTotalPeaks, MBTPeaks)

                    mQTPeaks<- NA
                    mQueryTotalPeaks<- c(mQueryTotalPeaks, mQTPeaks)

                    MBfor <- NA
                    MBformula<- c(MBformula, MBfor)

                    MBinchiK <- NA
                    MBinchiKEY <- c(MBinchiKEY, MBinchiK)

                    MBID <- NA
                    MBspectrumID<- c(MBspectrumID, MBID)

                    MBname <- NA
                    MBcompound_name <- c(MBcompound_name, MBname)

                    MBSpec <- NA
                    MBmirrorSpec <- c(MBmirrorSpec, MBSpec)

                    Src <- NA
                    Source <- c(Source, Src)
                }

            }
            else if (length(sps) > 1 && length(mbank_with_mz) == 1){

                #' Compare experimental spectra against MassBank
                res <- compareSpectra(sps, mbank_with_mz, ppm = 15)
                #' obtain MB spectra that matches the most with m/z MS2 spectra
                gx <- which(res == max(res))
                gx <- gx[1]
                sps <- sps[gx]
                df_peaklists <- peakdf(mbank_with_mz, sps[gx], ppmx)
                mbank_best_match <- mbank_with_mz
                #' if there are more than 2 peak matching
                if (!(is.null(df_peaklists))){


                    #' plotMirror
                    name_plotmirror <- paste(dir_name, x,"_spectra_vs_", mbank_best_match$accession, "_spectra.pdf", sep ="")
                    pdf(name_plotmirror)
                    plotSpectraMirror(sps, mbank_best_match, tolerance = 0.2,
                                        labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                        labelSrt = -30)
                    grid()
                    dev.off()

                    mbscore <- max(res)
                    MBmax_similarity<- c(MBmax_similarity, mbscore)

                    MBmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(mbank_best_match)[[1]])+nrow(peaksData(sps)[[1]]))
                    MBmzScore <- c(MBmzScore, MBmz)

                    MBint <- mean(1-(df_peaklists[,"diff"]/100))
                    MBintScore <- c(MBintScore, MBint)

                    MQMatPeaks <- nrow(df_peaklists)
                    MQMatchingPeaks <- c(MQMatchingPeaks, MQMatPeaks)


                    MBTPeaks <- nrow(peaksData(mbank_best_match)[[1]])
                    MBTotalPeaks<- c(MBTotalPeaks, MBTPeaks)

                    mQTPeaks<- nrow(peaksData(sps)[[1]])
                    mQueryTotalPeaks<- c(mQueryTotalPeaks, mQTPeaks)

                    MBfor <- mbank_best_match$formula
                    MBformula<- c(MBformula, MBfor)

                    MBinchiK <- mbank_best_match$inchikey
                    MBinchiKEY <- c(MBinchiKEY, MBinchiK)

                    MBID <- mbank_best_match$accession
                    MBspectrumID<- c(MBspectrumID, MBID)

                    MBname <- mbank_best_match$name
                    MBcompound_name <- c(MBcompound_name, MBname)

                    MBSpec <- str_replace(name_plotmirror, input_dir, "./")
                    MBmirrorSpec <- c(MBmirrorSpec, MBSpec)

                    Src <- "MassBank"
                    Source <- c(Source, Src)
                    }
                else{

                    mbscore <- NA
                    MBmax_similarity<- c(MBmax_similarity, mbscore)

                    MBmz <- NA
                    MBmzScore <- c(MBmzScore, MBmz)

                    MBint <- NA
                    MBintScore <- c(MBintScore, MBint)

                    MQMatPeaks <- NA
                    MQMatchingPeaks <- c(MQMatchingPeaks, MQMatPeaks)

                    MBTPeaks <- NA
                    MBTotalPeaks<- c(MBTotalPeaks, MBTPeaks)

                    mQTPeaks<- NA
                    mQueryTotalPeaks<- c(mQueryTotalPeaks, mQTPeaks)

                    MBfor <- NA
                    MBformula<- c(MBformula, MBfor)


                    MBinchiK <- NA
                    MBinchiKEY <- c(MBinchiKEY, MBinchiK)

                    MBID <- NA
                    MBspectrumID<- c(MBspectrumID, MBID)

                    MBname <- NA
                    MBcompound_name <- c(MBcompound_name, MBname)

                    MBSpec <- NA
                    MBmirrorSpec <- c(MBmirrorSpec, MBSpec)

                    Src <- NA
                    Source <- c(Source, Src)
                }
            }
            else if (length(sps) == 1 && length(mbank_with_mz) == 1){

                #' Compare experimental spectra against MassBank
                res <- compareSpectra(sps, mbank_with_mz, ppm = 15)
                mbank_best_match <- mbank_with_mz
                df_peaklists <- peakdf(mbank_best_match, sps, ppmx)
                if (!(is.null(df_peaklists))){


                    #' plotMirror
                    name_plotmirror <- paste(dir_name, x,"_spectra_vs_", mbank_best_match$accession, "_spectra.pdf", sep ="")
                    pdf(name_plotmirror)
                    plotSpectraMirror(sps, mbank_best_match, tolerance = 0.2,
                                        labels = label_fun, labelPos = 2, labelOffset = 0.2,
                                        labelSrt = -30)
                    grid()
                    dev.off()


                    mbscore <- max(res)
                    MBmax_similarity<- c(MBmax_similarity, mbscore)

                    MBmz <- (nrow(df_peaklists)*2)/(nrow(peaksData(mbank_best_match)[[1]])+nrow(peaksData(sps)[[1]]))
                    MBmzScore <- c(MBmzScore, MBmz)

                    MBint <- mean(1-(df_peaklists[,"diff"]/100))
                    MBintScore <- c(MBintScore, MBint)

                    MQMatPeaks <- nrow(df_peaklists)
                    MQMatchingPeaks <- c(MQMatchingPeaks, MQMatPeaks)

                    MBTPeaks <- nrow(peaksData(mbank_best_match)[[1]])
                    MBTotalPeaks<- c(MBTotalPeaks, MBTPeaks)

                    mQTPeaks<- nrow(peaksData(sps)[[1]])
                    mQueryTotalPeaks<- c(mQueryTotalPeaks, mQTPeaks)

                    MBfor <- mbank_best_match$formula
                    MBformula<- c(MBformula, MBfor)

                    MBinchiK <- mbank_best_match$inchikey
                    MBinchiKEY <- c(MBinchiKEY, MBinchiK)

                    MBID <- mbank_best_match$accession
                    MBspectrumID<- c(MBspectrumID, MBID)

                    MBname <- mbank_best_match$name
                    MBcompound_name <- c(MBcompound_name, MBname)

                    MBSpec <- str_replace(name_plotmirror, input_dir, "./")
                    MBmirrorSpec <- c(MBmirrorSpec, MBSpec)

                    Src <- "MassBank"
                    Source <- c(Source, Src)
                }
                else{

                    mbscore <- NA
                    MBmax_similarity<- c(MBmax_similarity, mbscore)

                    MBmz <- NA
                    MBmzScore <- c(MBmzScore, MBmz)

                    MBint <- NA
                    MBintScore <- c(MBintScore, MBint)

                    MQMatPeaks <- NA
                    MQMatchingPeaks <- c(MQMatchingPeaks, MQMatPeaks)

                    MBTPeaks <- NA
                    MBTotalPeaks<- c(MBTotalPeaks, MBTPeaks)

                    mQTPeaks<- NA
                    mQueryTotalPeaks<- c(mQueryTotalPeaks, mQTPeaks)

                    MBfor <- NA
                    MBformula<- c(MBformula, MBfor)


                    MBinchiK <- NA
                    MBinchiKEY <- c(MBinchiKEY, MBinchiK)

                    MBID <- NA
                    MBspectrumID<- c(MBspectrumID, MBID)

                    MBname <- NA
                    MBcompound_name <- c(MBcompound_name, MBname)

                    MBSpec <- NA
                    MBmirrorSpec <- c(MBmirrorSpec, MBSpec)

                    Src <- NA
                    Source <- c(Source, Src)
                }

            }
            else{

                mbscore <- NA
                MBmax_similarity<- c(MBmax_similarity, mbscore)

                MBmz <- NA
                MBmzScore <- c(MBmzScore, MBmz)

                MBint <- NA
                MBintScore <- c(MBintScore, MBint)

                MQMatPeaks <- NA
                MQMatchingPeaks <- c(MQMatchingPeaks, MQMatPeaks)

                MBTPeaks <- NA
                MBTotalPeaks<- c(MBTotalPeaks, MBTPeaks)

                mQTPeaks<- NA
                mQueryTotalPeaks<- c(mQueryTotalPeaks, mQTPeaks)

                MBfor <- NA
                MBformula<- c(MBformula, MBfor)

                MBinchiK <- NA
                MBinchiKEY <- c(MBinchiKEY, MBinchiK)


                MBID <- NA
                MBspectrumID<- c(MBspectrumID, MBID)

                MBname <- NA
                MBcompound_name <- c(MBcompound_name, MBname)

                MBSpec <- NA
                MBmirrorSpec <- c(MBmirrorSpec, MBSpec)

                Src <- NA
                Source <- c(Source, Src)

            }
        }
        
        df_mbank <- cbind(id_X, premz, rtmin, rtmax, rtmed, rtmean, MBmax_similarity, MBmzScore, MBintScore, MQMatchingPeaks, 
                         MBTotalPeaks, mQueryTotalPeaks, MBformula, MBinchiKEY, MBspectrumID, MBcompound_name, MBmirrorSpec, 
                         Source)
        write.csv(df_mbank, paste(input_dir, str_remove(paste(result_dir, "/spectral_dereplication/mbank.csv", sep = ""), "./"), sep = ""))


    }
    
}